## Imports

In [ ]:
import copy
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

### Helper function

In [ ]:
def calculate_regret(history : dict) :

    running_X = history["running_X"]
    running_Y = history["running_Y"]
    phi = history["phi"]
    running_X_best = history["running_X_best"]

    M, N = phi.shape
    regrets = []
    average_regrets = []

    online_loss = 0
    x_best_ = np.zeros(N)
    for i, (x, y, x_b) in tqdm(
        enumerate(zip(running_X, running_Y, running_X_best)),
        total = len(running_X)
        ) :
        online_loss += 0.5 * np.linalg.norm(y - phi @ x) ** 2

        x_best_ += x_b
        offline_loss = 0
        for j in range(i + 1) :
            offline_loss += 0.5 * np.linalg.norm(running_Y[j] - phi @ (x_best_ / (i+1)))

        # print(f"Online loss : {online_loss}, Offline Loss {offline_loss}, Regret : {online_loss - offline_loss}")
        regrets.append(online_loss - offline_loss)
        average_regrets.append((online_loss - offline_loss) / (i+1))

    history["regrets"] = regrets
    history["average_regrets"] = average_regrets

    return history

## HTP

In [ ]:
def ft_hard_thresholded_pl(eta, mu, phi, K, T) :
    """
    Follow the Hard-Thresholded Perturbed Leader algorithm.

    Inputs :
    eta : multiplicative factor for perturbation
    mu : some constant > 0, preferably != 1
    phi : shape(M, N)
    K : sparsity constant, number of non-zero elements
    T : timesteps to run the simulation for

    Returns :
    A dictionary containing `rewards`, `running_Y`, and `running_X`
    """

    # init
    M, N = phi.shape
    running_Y = []
    running_X = []
    running_X_best = []
    rewards = []
    support_x = np.random.permutation(N)[:K] # fixed support setting

    Y = np.zeros(M)
    gamma = np.random.normal(loc = 0.0, scale = 1.0, size = M) # constant

    for t in tqdm(range(1, T + 1)) :
        z = np.zeros(N)
        b = (Y + eta * gamma) / t

        # get x from oracle
        tau = int(np.log(t)) + 10
        for s in range(tau) :
            # abs? I think needed.
            L = np.argsort(np.abs(z + mu * phi.T @ (b - phi @ z)))[::-1][:K]
            z_new = np.linalg.lstsq(phi[:, L], b, rcond = None)[0]

            z = np.zeros(N)
            z[L] = z_new

        x = z

        # reveal y
        x_best = np.zeros(N)
        ###### very important
        x_best[support_x] = np.random.normal(loc = 0.0, scale = 1/5.64, size = K)
        noise = np.random.normal(scale = 1/1024, size = M)

        y = (phi @ x_best) + noise

        # get reward
        reward = np.dot(y, phi @ x) - 0.5 * np.linalg.norm(phi @ x) ** 2
        Y = Y + y

        rewards.append(reward)
        running_X.append(x)
        running_Y.append(y)
        running_X_best.append(x_best)

    accumulated_rewards = copy.deepcopy(rewards)
    for i in range(1, len(accumulated_rewards)) :
        accumulated_rewards[i] += accumulated_rewards[i-1]

    for i in range(len(accumulated_rewards)) :
        accumulated_rewards[i] /= i+1

    return {
        "phi" : phi,
        "rewards" : rewards,
        "average_rewards" : accumulated_rewards,
        "running_X" : running_X,
        "running_X_best" : running_X_best,
        "running_Y" : running_Y
    }


## CoSaMP

In [ ]:
def ft_cosamp_pl(eta, phi, K, T) :
    """
    Follow the CoSaMPed Perturbed Leader algorithm.

    Inputs :
    eta : multiplicative factor for perturbation
    phi : shape(M, N)
    K : sparsity constant, number of non-zero elements
    T : timesteps to run the simulation for

    Returns :
    A dictionary containing `rewards`, `running_Y`, and `running_X`
    """

    # init
    M, N = phi.shape
    running_Y = []
    running_X = []
    running_X_best = []
    rewards = []
    support_x = np.random.permutation(N)[:K] # fixed support setting

    Y = np.zeros(M)
    gamma = np.random.normal(loc = 0.0, scale = 1.0, size = M) # constant

    for t in tqdm(range(1, T + 1)) :
        z = np.zeros(N)
        b = (Y + eta * gamma) / t

        # get x from oracle
        tau = int(np.log(t) + 10)
        for s in range(tau) :
            # abs? I think needed.
            sup1 = np.nonzero(z)
            sup2 = np.argsort(np.abs(phi.T @ (b - phi @ z)))[::-1][:2*K]

            L = np.union1d(sup1, sup2)
            u = np.linalg.lstsq(phi[:, L], b, rcond = None)[0]

            topK = np.argsort(np.abs(u))[::-1][:K]
            z = np.zeros(N)
            z[topK] = u[topK]

        x = z

        # reveal y
        x_best = np.zeros(N)
        x_best[support_x] = np.random.normal(loc = 0.0, scale = 1/5.64, size = K)
        noise = np.random.normal(scale = 1/128, size = M)

        y = (phi @ x_best) + noise

        # get reward
        reward = np.dot(y, phi @ x) - 0.5 * np.linalg.norm(phi @ x) ** 2
        Y = Y + y

        rewards.append(reward)
        running_X.append(x)
        running_Y.append(y)
        running_X_best.append(x_best)

    accumulated_rewards = copy.deepcopy(rewards)
    for i in range(1, len(accumulated_rewards)) :
        accumulated_rewards[i] += accumulated_rewards[i-1]

    for i in range(len(accumulated_rewards)) :
        accumulated_rewards[i] /= i+1

    return {
        "phi" : phi,
        "rewards" : rewards,
        "average_rewards" : accumulated_rewards,
        "running_X" : running_X,
        "running_X_best" : running_X_best,
        "running_Y" : running_Y
    }

##IHTP

In [ ]:
def ft_iter_hard_thresholded_pl(eta, mu, phi, K, T) :
    """
    Follow the Iterative-Hard-Thresholded Perturbed Leader algorithm.

    Inputs :
    eta : multiplicative factor for perturbation
    mu : some constant > 0, preferably != 1
    phi : shape(M, N)
    K : sparsity constant, number of non-zero elements
    T : timesteps to run the simulation for

    Returns :
    A dictionary containing `rewards`, `running_Y`, and `running_X`
    """

    # init
    M, N = phi.shape
    running_Y = []
    running_X = []
    running_X_best = []
    rewards = []
    support_x = np.random.permutation(N)[:K] # fixed support setting

    Y = np.zeros(M)
    gamma = np.random.normal(loc = 0.0, scale = 1.0, size = M) # constant

    for t in tqdm(range(1, T + 1)) :
        z = np.zeros(N)
        b = (Y + eta * gamma) / t

        # get x from oracle
        tau = int(np.log(t)) + 10
        for s in range(tau) :
            # abs? I think needed.
            r = z + phi.T @ (b - phi @ z)
            L = np.argsort(np.abs(r))[::-1][K]

            z = np.zeros(N)
            z[L] = r[L]

        x = z

        # reveal y
        x_best = np.zeros(N)
        ###### very important
        x_best[support_x] = np.random.normal(loc = 0.0, scale = 1/5.64, size = K)
        #####################
        noise = np.random.normal(scale = 1/1024, size = M)

        y = (phi @ x_best) + noise

        # get reward
        reward = np.dot(y, phi @ x) - 0.5 * np.linalg.norm(phi @ x) ** 2
        Y = Y + y

        rewards.append(reward)
        running_X.append(x)
        running_Y.append(y)
        running_X_best.append(x_best)

    accumulated_rewards = copy.deepcopy(rewards)
    for i in range(1, len(accumulated_rewards)) :
        accumulated_rewards[i] += accumulated_rewards[i-1]

    for i in range(len(accumulated_rewards)) :
        accumulated_rewards[i] /= i+1

    for i in range(len(accumulated_rewards)) :
        accumulated_rewards[i] /= i+1

    return {
        "phi" : phi,
        "rewards" : rewards,
        "average_rewards" : accumulated_rewards,
        "running_X" : running_X,
        "running_X_best" : running_X_best,
        "running_Y" : running_Y
    }

## Subspace Pursuit

In [ ]:
def residual(y, A) :
    yp = (A @ np.linalg.pinv(A)) @ y
    res = y - yp
    return res


def ft_subspace_pursuit_pl(eta, phi, K, T) :
    """
    Follow the Subspace Pursuit Perturbed Leader algorithm.

    Inputs :
    eta : multiplicative factor for perturbation
    phi : shape(M, N)
    K : sparsity constant, number of non-zero elements
    T : timesteps to run the simulation for

    Returns :
    A dictionary containing `rewards`, `running_Y`, and `running_X`
    """

    # init
    M, N = phi.shape
    running_Y = []
    running_X = []
    running_X_best = []
    rewards = []
    support_x = np.random.permutation(N)[:K] # fixed support setting

    Y = np.zeros(M)
    gamma = np.random.normal(loc = 0.0, scale = 1.0, size = M) # constant

    for t in tqdm(range(1, T + 1)) :
        b = (Y + eta * gamma) / t

        tl = np.argsort(np.abs(phi.T @ b))[::-1][:K]
        r = residual(b, phi[:, tl])

        # get x from oracle
        tau = int(np.log(t) + 10)
        for s in range(tau) :

            sup = np.argsort(np.abs(phi.T @ r))[::-1][:K]
            tl_ = np.union1d(tl, sup)

            x_p = np.linalg.pinv(phi[:, tl_]) @ b

            tl = np.argsort(np.abs(x_p))[::-1][:K]
            r = residual(b, phi[:, tl])

        x = np.zeros(N)
        x[tl] = np.linalg.pinv(phi[:, tl]) @ b

        # reveal y
        x_best = np.zeros(N)
        x_best[support_x] = np.random.normal(loc = 0.0, scale = 1/5.818, size = K)
        noise = np.random.normal(scale = 1/128, size = M)

        y = (phi @ x_best) + noise

        # get reward
        reward = np.dot(y, phi @ x) - 0.5 * np.linalg.norm(phi @ x) ** 2
        Y = Y + y

        rewards.append(reward)
        running_X.append(x)
        running_Y.append(y)
        running_X_best.append(x_best)

    accumulated_rewards = copy.deepcopy(rewards)
    for i in range(1, len(accumulated_rewards)) :
        accumulated_rewards[i] += accumulated_rewards[i-1]

    for i in range(len(accumulated_rewards)) :
        accumulated_rewards[i] /= i+1

    return {
        "phi" : phi,
        "rewards" : rewards,
        "average_rewards" : accumulated_rewards,
        "running_X" : running_X,
        "running_X_best" : running_X_best,
        "running_Y" : running_Y
    }

## Testing

In [ ]:
np.random.seed(42)

M = 128 # try 256 too
N = 512
K = 32 # keep <= 40
T = 1000 # time steps

eta = 0.2 # try others
mu = 0.3

phi = np.random.normal(loc = 0.0, scale = 1/np.sqrt(M), size = (M, N))

In [ ]:
history = ft_cosamp_pl(eta, phi, K, T)
alg = "cosamp"
history = calculate_regret(history)

df = pd.DataFrame(
    {
        "Regret" : history["regrets"],
        "Average Regret" : history["average_regrets"],
        "Reward" : history["rewards"],
        "Average Reward" : history["average_rewards"]
    }
)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (8, 5), dpi = 200)

a = sns.lineplot(df[["Regret"]][:200], ax = ax[0][0]);
b = sns.lineplot(df[["Reward"]][:200], ax = ax[0][1]);
c = sns.lineplot(df[["Average Regret"]][:200], ax = ax[1][0]);
f = sns.lineplot(df[["Average Reward"]][:200], ax = ax[1][1]);

plt.tight_layout()
plt.savefig(f"logs/{alg}_M{M}_N{N}_K{K}_T{T}_eta{eta}_mu{mu}_scale5.818.png")